<a href="https://colab.research.google.com/github/DGkurnia/sentiment/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Nama : Derfansyah Guswiranata Kurnia
- Iddicoding: dgkurnia
- email dicoding: weerakurnia@gmai.com

#Insiasi

In [82]:
#Perpustakaan tahap awal
!pip install selenium beautifulsoup4 pandas

In [83]:
#Impor Bagian awal
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
#modifikasi percepatan
import asyncio
import aiohttp
from concurrent.futures import ProcessPoolExecutor

In [84]:
#Persiapan pembuatan ulasan
import requests  # Mengirim permintaan HTTP
#mode' tanpa kepala'
from selenium.webdriver.chrome.service import Service #layanan
from selenium.webdriver.chrome.options import Options #pilihan

In [85]:
#Next Import
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks

## Sastrawi

In [86]:
#Instalasi sastrawi
!pip install sastrawi

In [87]:
#Impor Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia
#ke wordcloud

In [88]:
#seksi wordcloud
!pip install wordcloud
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
# ke NITK

### Pembaruan rutin

In [89]:
#pembaruan
!apt-get update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [90]:
#chromedriver
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0ubuntu2.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 55 not upgraded.


In [91]:
#impor file tambahan (google drive)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Perpustakaan lain untuk analisis

In [92]:
#Seksi Io dan lain-lain
import os
from io import StringIO
import concurrent.futures #untuk pemrograman paralel
from concurrent.futures import ThreadPoolExecutor

In [93]:
#Interpolasi untuk perbaikan: Grafik
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
#perpustakaan untuk interpolasi
from scipy.interpolate import interp1d

In [94]:
#NITK
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [100]:
#Bagian lain
import csv
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, f1_score

### Persiapan untuk analisis

In [101]:
#pandas
import pandas as pd
import time

In [102]:
#Ekstraksi
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer  #sentimen
#pemisahan
from sklearn.model_selection import train_test_split as ttsplit
#ke klasifikasi

In [103]:
#klasifikasi randomforest
from sklearn.ensemble import RandomForestClassifier as rfclasifi #teknik 'randomfprest'
from sklearn.tree import DecisionTreeClassifier #teknik 'keputusan'
from sklearn.svm import SVC #teknik SVC
from sklearn.linear_model import LogisticRegression

## Pengambilan ulasan dari situs Tokopedia

In [104]:
#persiapan sesi
session = requests.Session()

In [105]:
#Tahap awal pengambilan
urllist = [
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-jazlyn-black-jazlyn-black-s/review",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-gribson-rayon-navy-s/review",
    "https://www.tokopedia.com/erigo/kemeja-unisex-erigo-short-shirt-lavrenti-white-lavrenti-white-s/review"
]

Ada tiga ulasan yang dipakai untuk analisis

In [120]:
#Persiapan untuk proses pengambilan list: Tahap Extraksi HTM secara asinkronus
async def fetch_html(url):
    response = await asyncio.get_event_loop().run_in_executor(None, session.get, url)
    response.raise_for_status()  # Penanda eror
    return response.content
#ke scrapping

In [121]:
#Proses utama 'ekstraksi'
async def review_scrapper(url):
    html_content = await fetch_html(url)
    soup = BeautifulSoup(html_content, 'html.parser')

    # ekstraksi ulasan
    results = []  # Bentuk awal
    for rev in soup.find_all('div', class_='review-class'):
        teks = rev.find('div', class_='review-content').text.strip()
        results.append(teks)  # Proses ekstraksi
    return results  # Return the list of reviews
#ke persiapan url

In [122]:
#persiapan asinkronus untuk pemrosesan url
def process_urls(chunk):
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    # Tugas asinkronus
    tasks = [review_scrapper(url) for url in chunk]
    results = loop.run_until_complete(asyncio.gather(*tasks))

    # Pemerataan hasil
    return [item for sublist in results for item in sublist]
#ke fungsi utama

Pengambilan nilai ulasan per potongan

In [123]:
#fungsi pemotongan analisis
def chunk_and_execute(urls, chunk_size=14, output_prefix='intermediate_reviews'):
    checkpoint_file = f'{output_prefix}_checkpoint.txt'

    # Penyimpanan ekstraksi
    start_index = 0
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            start_index = int(f.read().strip())

    with open(f'{output_prefix}_final.csv', 'a', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Review']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Penulisan kepala
        if start_index == 0:
            writer.writeheader()

        num_chunks = (len(urls) + chunk_size - 1) // chunk_size

        with ProcessPoolExecutor(max_workers=10) as executor:
            for i in range(start_index, num_chunks):
                start_idx = i * chunk_size
                end_idx = min((i + 1) * chunk_size, len(urls))

                chunk_url_list = urls[start_idx:end_idx]
                results = executor.submit(process_urls, chunk_url_list).result()

                # Write each review to CSV file
                for review in results:
                    writer.writerow({'Review': review})

                # Penyimpanan indeks
                with open(checkpoint_file, 'w') as f:
                    f.write(str(i + 1))  # Simpan indeks lain
#ke eksekusi

In [ ]:
#penyatuan ke hasil akhir
hslakhir = chunk_and_execute(urllist)
#ke penyetelan data csv

totulas adalah **'total ulasan'** yang dipakai untuk analisis hasi akhir **(hslakhir)**

### Persiapan data-jadi
Inspeksi hasil akhir

In [ ]:
#inspeksi hasil akhir
print(hslakhir)

In [ ]:
#persiapan df untuk pembersihan
dfori = pd.DataFrame(hslakhir, columns=['Ulasan'])
dfkopi = dfori.copy() #Jangan pakai data asli
dfkopi.to_csv('tokopedia_reviews.csv', index=False)

data yang dipakai adala **'dfkopi'**

In [ ]:
#Inspeksi dataset
dfinfo = dfkopi.info()
print(dfinfo)

# Tahap awal (pembersihan)

In [ ]:
#persiapan untuk pembersihan


In [ ]:
#Persiapan tahap awal


In [ ]:
#Pembersihan tahap lanjut
